In [62]:
import pandas as pd 
import numpy as np
import matplotlib as lyb
import seaborn as sns


In [63]:
data=pd.read_csv("./Data-cleaning-for-beginners-using-pandas.csv")
data.head()



,Index,Age,Salary,Rating,Location,Established,Easy Apply
0,0,44.0,$44k-$99k,5.4,"India,In",1999,TRUE
1,1,66.0,$55k-$66k,3.5,"New York,Ny",2002,TRUE
2,2,NaN,$77k-$89k,-1.0,"New York,Ny",-1,-1
3,3,64.0,$44k-$99k,4.4,India In,1988,-1
4,4,25.0,$44k-$99k,6.4,Australia Aus,2002,-1


In [64]:
def cleanData(data):

    # min and max salry 
    data["minSalary"]=(data["Salary"]
                       .str.replace("$","",regex=False)
                       .str.replace("k","",regex=False)
                       .str.split("-",expand=True)[0]
                       .astype(int)*1000


                       
                       )
    
    data["maxSalary"]=(data["Salary"]
                       .str.replace("$","",regex=False)
                       .str.replace("k","",regex=False)
                       .str.split("-",expand=True)[1]
                       .astype(int)*1000

    )

    # fix location

    data["Location"]=data["Location"].str.replace(" ",",",1)

    data[["city","country"]]=data["Location"].str.rsplit(",",n=1,expand=True)

    data["city"]=data["city"].str.replace(","," ",1)


    # reeplace -1 to NaN

    data["Easy Apply"]=data["Easy Apply"].str.replace("-1","UNKNOWN")
    data["Rating"]=data["Rating"].replace(-1,np.nan)
    data["Established"]=data["Established"].replace(-1,np.nan)

        # Create experiance level

    data["Experience Level"]=np.where(data["Age"]<30,"Junior",
                                      np.where(data["Age"]<=50,"mid","senior")
                                      )
    

    # Outliner Removal

    Q1=data["Rating"].quantile(0.25)
    Q3=data["Rating"].quantile(0.75)

    IRQ=Q3-Q1

    lower=Q1-1.5*IRQ
    upper=Q3+1.5*IRQ

    data=data[(data["Rating"]>=lower) & (data["Rating"]<=upper)]


    

    data=data.reset_index(drop=True)

    return data

cleanDataSet=cleanData(data)
cleanDataSet.head()

,Index,Age,Salary,Rating,Location,Established,Easy Apply,minSalary,maxSalary,city,country,Experience Level
0,0,44.0,$44k-$99k,5.4,"India,In",1999.0,TRUE,44000,99000,India,In,mid
1,1,66.0,$55k-$66k,3.5,"New,York,Ny",2002.0,TRUE,55000,66000,New York,Ny,senior
2,3,64.0,$44k-$99k,4.4,"India,In",1988.0,UNKNOWN,44000,99000,India,In,senior
3,4,25.0,$44k-$99k,6.4,"Australia,Aus",2002.0,UNKNOWN,44000,99000,Australia,Aus,Junior
4,5,44.0,$77k-$89k,1.4,"India,In",1999.0,TRUE,77000,89000,India,In,mid


Q1. Show top 5 cities with the highest number of job listings.

In [65]:
cleanDataSet["city"].value_counts().head()

city
New York     10
India         9
Australia     5
Name: count, dtype: int64

Q2. What is the average minimum salary and average maximum salary for each country?

In [70]:
avgMin=cleanDataSet.groupby("country")["minSalary"].mean()
avgMax=cleanDataSet.groupby("country")['maxSalary'].mean()
print("Average min Salary: ",avgMin)
print("Average max Salary: ",avgMax)

Average min Salary:  country
Aus    51800.000000
In     37333.333333
Ny     46200.000000
Name: minSalary, dtype: float64
Average max Salary:  country
Aus    97200.000000
In     80222.222222
Ny     92400.000000
Name: maxSalary, dtype: float64


DA Step 3 – Age & Experience Insights
Q3. How many Junior, Mid, and Senior level positions are there

In [72]:
cleanDataSet['Experience Level'].value_counts()

Experience Level
senior    11
mid        8
Junior     5
Name: count, dtype: int64

EDA Step 4 — Rating Insights
Q4. What is the average rating for each city?

In [74]:
AvgRating=cleanDataSet.groupby('city')['Rating'].mean().round(2)
print("Average Rating of each city : ",AvgRating)

Average Rating of each city :  city
Australia    3.36
India        4.54
New York     4.51
Name: Rating, dtype: float64


EDA Step 5 — Salary Insights (Advanced)
Q5. Find the top 5 highest-paying cities based on average MaxSalary.

In [78]:
top5=cleanDataSet.groupby('city')['maxSalary'].mean().sort_values(ascending=False).head().round(2)
print(top5)

city
Australia    97200.00
New York     92400.00
India        80222.22
Name: maxSalary, dtype: float64


NEXT EDA STEP (Correlation Analysis)
Q6. Find correlation between:
Age,Rating,minSalary,maxSalary

In [82]:
colleration=cleanDataSet[["Age","Rating","minSalary","maxSalary"]].corr().round(1)
print(colleration)

           Age  Rating  minSalary  maxSalary
Age        1.0    -0.3        0.0        0.1
Rating    -0.3     1.0       -0.3       -0.2
minSalary  0.0    -0.3        1.0        0.5
maxSalary  0.1    -0.2        0.5        1.0
